In [29]:
# TESTE DE CHAMADA DO SERVIÇO NO ENDEREÇO: /v5/market/orderbook

import sys
import os
# Adiciona o diretório raiz do projeto ao sys.path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(),'..', 'src')))

from corretoras.funcoes_bybit import cliente

# Configurações
simbolo = 'BTCUSDT'
# intervalo = '15'

# df = busca_velas(simbolo, intervalo, [9, 21])
# df.tail()

order_book = cliente.get_orderbook(category='linear', symbol=simbolo, limit=100)
order_book

{'retCode': 0,
 'retMsg': 'OK',
 'result': {'s': 'BTCUSDT',
  'b': [['111577.2', '0.465'],
   ['111575.8', '0.001'],
   ['111575.5', '0.002'],
   ['111575.4', '0.001'],
   ['111574.4', '0.002'],
   ['111573.4', '0.001'],
   ['111573.3', '0.003'],
   ['111573', '0.001'],
   ['111571.3', '0.008'],
   ['111571.1', '0.002'],
   ['111570.5', '0.001'],
   ['111570', '0.002'],
   ['111569.4', '0.037'],
   ['111569.2', '0.001'],
   ['111568.9', '0.002'],
   ['111568.7', '0.001'],
   ['111568.5', '0.016'],
   ['111567.6', '0.37'],
   ['111567.2', '0.001'],
   ['111567.1', '0.022'],
   ['111566.7', '0.002'],
   ['111566.1', '0.014'],
   ['111566', '0.094'],
   ['111565.8', '0.008'],
   ['111565.5', '0.002'],
   ['111565', '0.003'],
   ['111564.9', '0.001'],
   ['111564.7', '0.08'],
   ['111564.4', '0.002'],
   ['111564.2', '0.37'],
   ['111564', '0.001'],
   ['111563.6', '0.001'],
   ['111563.5', '0.001'],
   ['111563.3', '0.002'],
   ['111563.2', '0.004'],
   ['111562.7', '0.009'],
   ['111562.

In [30]:
# Liquidez por bucket com:
# - bucket dinâmico (pct/usd/ticks)
# - arredondamento "1-2-5" para número bonito (ex.: 11.2 -> 10; 1.12 -> 1)
# - alinhamento ao tick e sem NaN/0 na visualização por padrão

import pandas as pd
import numpy as np

# Parâmetros
bucket_mode = 'pct'      # 'pct' (bps), 'usd', 'ticks'
bucket_bps = 1.0         # 1 bps = 0,01% (usado se bucket_mode == 'pct')
bucket_usd_param = 10.0  # usado se bucket_mode == 'usd'
ticks_param = 5          # usado se bucket_mode == 'ticks'

nice_bucket = True       # usar arredondamento 1-2-5
nice_mode = 'floor'      # 'floor' | 'round' | 'ceil'
show_empty_buckets = False  # True = mostra buckets vazios (preenche com 0/NaN)
fill_empty_minmax_with_bounds = False  # se mostrar vazios, preencher min/max com limites do bucket

# Extrai do orderbook
bids = order_book['result']['b']
asks = order_book['result']['a']

def infer_tick_size_from_levels(bids_levels, asks_levels):
    def count_decimals(p_str):
        return len(p_str.split('.')[1]) if '.' in p_str else 0
    dec_b = [count_decimals(p) for p, _ in bids_levels] if bids_levels else [0]
    dec_a = [count_decimals(p) for p, _ in asks_levels] if asks_levels else [0]
    max_dec = max(dec_b + dec_a)
    return 10 ** (-max_dec) if max_dec > 0 else 1.0

def nice_125(x, mode='floor'):
    if x <= 0:
        return x
    exp = np.floor(np.log10(x))
    frac = x / (10 ** exp)
    bases = [1, 2, 5]
    if mode == 'floor':
        if frac >= 5:
            n = 5
        elif frac >= 2:
            n = 2
        else:
            n = 1
    elif mode == 'ceil':
        if frac <= 1:
            n = 1
        elif frac <= 2:
            n = 2
        elif frac <= 5:
            n = 5
        else:
            n = 10
    else:  # 'round' aproximado
        if frac < 1.5:
            n = 1
        elif frac < 3.5:
            n = 2
        elif frac < 7.5:
            n = 5
        else:
            n = 10
    return n * (10 ** exp)

tick_size = infer_tick_size_from_levels(bids, asks)

# DataFrames
df_bids = pd.DataFrame(bids, columns=['price', 'quantity'])
df_bids['price'] = df_bids['price'].astype(float)
df_bids['quantity'] = df_bids['quantity'].astype(float)
df_bids['side'] = 'BID'

df_asks = pd.DataFrame(asks, columns=['price', 'quantity'])
df_asks['price'] = df_asks['price'].astype(float)
df_asks['quantity'] = df_asks['quantity'].astype(float)
df_asks['side'] = 'ASK'

df_orderbook = pd.concat([df_bids, df_asks], ignore_index=True)

best_bid = df_bids['price'].max()
best_ask = df_asks['price'].min()
mid = (best_bid + best_ask) / 2.0
spread = best_ask - best_bid

# Define bucket (USD)
if bucket_mode == 'pct':
    raw_bucket_usd = mid * (bucket_bps / 10000.0)
elif bucket_mode == 'usd':
    raw_bucket_usd = float(bucket_usd_param)
elif bucket_mode == 'ticks':
    raw_bucket_usd = max(ticks_param * tick_size, tick_size)
else:
    raw_bucket_usd = mid * 0.0001  # fallback

if nice_bucket:
    raw_bucket_usd = nice_125(raw_bucket_usd, mode=nice_mode)

# Ajuste ao tick (múltiplo inteiro de tick_size)
bucket_ticks = max(1, int(np.floor(raw_bucket_usd / tick_size)))
bucket_usd = bucket_ticks * tick_size

# Precisão de rótulo conforme tick
label_decimals = max(0, int(round(-np.log10(tick_size)))) if tick_size < 1 else 0

# Evita erros de ponto flutuante usando aritmética em ticks
df_orderbook['price_ticks'] = np.rint(df_orderbook['price'] / tick_size).astype(int)
bucket_idx = df_orderbook['price_ticks'] // bucket_ticks
df_orderbook['bucket_floor_ticks'] = bucket_idx * bucket_ticks
df_orderbook['bucket_floor_price'] = df_orderbook['bucket_floor_ticks'] * tick_size

df_orderbook['price_range_label'] = df_orderbook['bucket_floor_price'].apply(
    lambda x: f"{x:.{label_decimals}f} - {(x + bucket_usd):.{label_decimals}f}"
)

# Agrupa
liquidity_by_range = df_orderbook.groupby(
    ['bucket_floor_ticks', 'price_range_label', 'side']
).agg(
    Total_Quantity=('quantity', 'sum'),
    Min_Price=('price', 'min'),
    Max_Price=('price', 'max'),
    Order_Count=('price', 'count'),
).reset_index()

# Completa buckets vazios (opcional)
if show_empty_buckets:
    all_buckets = np.sort(liquidity_by_range['bucket_floor_ticks'].unique())
    all_sides = ['BID', 'ASK']
    complete_index = pd.MultiIndex.from_product([all_buckets, all_sides],
                                                names=['bucket_floor_ticks', 'side'])
    complete_df = pd.DataFrame(index=complete_index).reset_index()
    complete_df = complete_df.merge(
        liquidity_by_range.drop(columns=['price_range_label']),
        on=['bucket_floor_ticks', 'side'],
        how='left'
    )
    # Recria label
    complete_df['bucket_floor_price'] = complete_df['bucket_floor_ticks'] * tick_size
    complete_df['price_range_label'] = complete_df['bucket_floor_price'].apply(
        lambda x: f"{x:.{label_decimals}f} - {(x + bucket_usd):.{label_decimals}f}"
    )
    # Preenche vazios
    complete_df['Total_Quantity'] = complete_df['Total_Quantity'].fillna(0)
    complete_df['Order_Count'] = complete_df['Order_Count'].fillna(0).astype(int)
    if fill_empty_minmax_with_bounds:
        complete_df['Min_Price'] = complete_df['Min_Price'].fillna(complete_df['bucket_floor_price'])
        complete_df['Max_Price'] = complete_df['Max_Price'].fillna(complete_df['bucket_floor_price'] + bucket_usd)
    liquidity_by_range = complete_df[['price_range_label','side','Total_Quantity','Min_Price','Max_Price','Order_Count','bucket_floor_ticks']]

# Ordena: ASKs (baixo->alto), depois BIDs (alto->baixo)
asks_df = liquidity_by_range[liquidity_by_range['side'] == 'ASK'].sort_values('bucket_floor_ticks', ascending=True)
bids_df = liquidity_by_range[liquidity_by_range['side'] == 'BID'].sort_values('bucket_floor_ticks', ascending=False)

liquidity_consolidated_full = pd.concat([asks_df, bids_df], ignore_index=True)
liquidity_consolidated_full = liquidity_consolidated_full[['price_range_label','side','Total_Quantity','Min_Price','Max_Price','Order_Count']]

# Print
print("\n" + "="*80)
mode_str = f"PCT | bucket≈{bucket_usd:.{label_decimals}f} USD ({bucket_bps} bps)" if bucket_mode=='pct' else f"{bucket_mode.upper()} | bucket≈{bucket_usd:.{label_decimals}f} USD"
print(f"📊 LIQUIDEZ POR BUCKET ({mode_str})")
print("="*80)
print(liquidity_consolidated_full.to_string(index=False))

print("\n" + "="*80)
print("📈 RESUMO GERAL")
print("="*80)
total_bids = df_bids['quantity'].sum()
total_asks = df_asks['quantity'].sum()
print(f"Total BID: {total_bids:.3f}")
print(f"Total ASK: {total_asks:.3f}")
print(f"Melhor BID: {best_bid:.{label_decimals}f}")
print(f"Melhor ASK: {best_ask:.{label_decimals}f}")
print(f"Spread: {(best_ask-best_bid):.{label_decimals}f}")
print(f"Bid/Ask Ratio: {total_bids/total_asks:.2f}")

liquidity_consolidated_full


📊 LIQUIDEZ POR BUCKET (PCT | bucket≈10.0 USD (1.0 bps))
  price_range_label side  Total_Quantity  Min_Price  Max_Price  Order_Count
111570.0 - 111580.0  ASK           7.674   111577.3   111579.4            9
111580.0 - 111590.0  ASK           8.541   111580.0   111589.8           59
111590.0 - 111600.0  ASK           8.928   111590.0   111593.8           32
111570.0 - 111580.0  BID           0.489   111570.0   111577.2           12
111560.0 - 111570.0  BID           2.195   111560.0   111569.4           33
111550.0 - 111560.0  BID          11.543   111551.4   111559.8           55

📈 RESUMO GERAL
Total BID: 14.227
Total ASK: 25.143
Melhor BID: 111577.2
Melhor ASK: 111577.3
Spread: 0.1
Bid/Ask Ratio: 0.57


,price_range_label,side,Total_Quantity,Min_Price,Max_Price,Order_Count
0,111570.0 - 111580.0,ASK,7.674,111577.3,111579.4,9
1,111580.0 - 111590.0,ASK,8.541,111580.0,111589.8,59
2,111590.0 - 111600.0,ASK,8.928,111590.0,111593.8,32
3,111570.0 - 111580.0,BID,0.489,111570.0,111577.2,12
4,111560.0 - 111570.0,BID,2.195,111560.0,111569.4,33
5,111550.0 - 111560.0,BID,11.543,111551.4,111559.8,55


In [31]:
# TESTE DE CHAMADA DO SERVIÇO NO ENDEREÇO: /v5/market/rpi_orderbook

from pybit.unified_trading import HTTP

session = HTTP(testnet=False)

def get_rpi_orderbook(symbol: str, category: str = "linear", limit: int = 50):
    endpoint = "https://api.bybit.com/v5/market/rpi_orderbook"
    params = {"symbol": symbol, "category": category, "limit": limit}
    response = session._submit_request(
        method="GET",
        path=endpoint,
        query=params
    )
    return response

rpi_orderbook = get_rpi_orderbook("BTCUSDT", limit=50)
rpi_orderbook

{'retCode': 0,
 'retMsg': 'OK',
 'result': {'s': 'BTCUSDT',
  'a': [['111577.30', '6.595', '0.000'],
   ['111577.40', '0.920', '0.000'],
   ['111577.50', '0.145', '0.000'],
   ['111577.80', '0.001', '0.000'],
   ['111578.70', '0.005', '0.000'],
   ['111578.80', '0.002', '0.000'],
   ['111578.90', '0.013', '0.000'],
   ['111579.30', '0.002', '0.000'],
   ['111579.40', '0.001', '0.000'],
   ['111580.00', '0.002', '0.000'],
   ['111580.30', '0.022', '0.000'],
   ['111580.40', '0.009', '0.000'],
   ['111580.80', '0.015', '0.000'],
   ['111581.20', '0.001', '0.000'],
   ['111581.60', '0.006', '0.000'],
   ['111581.70', '0.012', '0.000'],
   ['111581.90', '0.395', '0.000'],
   ['111582.00', '0.003', '0.000'],
   ['111582.20', '0.009', '0.000'],
   ['111582.30', '0.006', '0.000'],
   ['111582.50', '0.070', '0.000'],
   ['111583.00', '0.001', '0.000'],
   ['111583.70', '0.008', '0.000'],
   ['111583.90', '0.501', '0.000'],
   ['111584.00', '0.008', '0.000'],
   ['111584.10', '0.014', '0.000'],

In [32]:
import pandas as pd
import numpy as np

# Parâmetros de bucket
bucket_mode = 'pct'     # 'pct' (bps), 'usd', 'ticks'
bucket_bps = 1.0        # 1 bps = 0,01% (se bucket_mode == 'pct')
bucket_usd_param = 10.0 # se bucket_mode == 'usd'
ticks_param = 5         # se bucket_mode == 'ticks'
nice_bucket = True      # arredondar para 1–2–5
nice_mode = 'floor'     # 'floor' | 'round' | 'ceil'
show_empty_buckets = False  # mostra ou não buckets sem ordens

def infer_tick_size_from_levels(bids_levels, asks_levels):
    def count_decimals(p_str):
        return len(p_str.split('.')[1]) if isinstance(p_str, str) and '.' in p_str else 0
    dec_b = [count_decimals(p) for p, *_ in bids_levels] if bids_levels else [0]
    dec_a = [count_decimals(p) for p, *_ in asks_levels] if asks_levels else [0]
    max_dec = max(dec_b + dec_a)
    return 10 ** (-max_dec) if max_dec > 0 else 1.0

def nice_125(x, mode='floor'):
    if x <= 0: return x
    exp = np.floor(np.log10(x))
    frac = x / (10 ** exp)
    if mode == 'floor':
        n = 5 if frac >= 5 else (2 if frac >= 2 else 1)
    elif mode == 'ceil':
        n = 1 if frac <= 1 else (2 if frac <= 2 else (5 if frac <= 5 else 10))
    else:  # round
        n = 1 if frac < 1.5 else (2 if frac < 3.5 else (5 if frac < 7.5 else 10))
    return n * (10 ** exp)

def to_df_from_levels(levels, side):
    if not levels:
        return pd.DataFrame(columns=['price','quantity','side'])
    df = pd.DataFrame(levels)
    df = df.iloc[:, :2]           # pega só preço e quantidade
    df.columns = ['price','quantity']
    df['price'] = df['price'].astype(float)
    df['quantity'] = df['quantity'].astype(float)
    df['side'] = side
    return df

# Usa a resposta do RPI: 'rpi_orderbook'
bids_levels = rpi_orderbook['result']['b']
asks_levels = rpi_orderbook['result']['a']

tick_size = infer_tick_size_from_levels(bids_levels, asks_levels)
label_decimals = max(0, int(round(-np.log10(tick_size)))) if tick_size < 1 else 0

df_bids = to_df_from_levels(bids_levels, 'BID')
df_asks = to_df_from_levels(asks_levels, 'ASK')
df_orderbook = pd.concat([df_bids, df_asks], ignore_index=True)

if df_bids.empty or df_asks.empty:
    # Se um dos lados vier vazio no RPI, ajuste mid/spread de forma robusta
    best_bid = df_bids['price'].max() if not df_bids.empty else (df_asks['price'].min() - tick_size)
    best_ask = df_asks['price'].min() if not df_asks.empty else (df_bids['price'].max() + tick_size)
else:
    best_bid = df_bids['price'].max()
    best_ask = df_asks['price'].min()
mid = (best_bid + best_ask) / 2.0
spread = max(0.0, best_ask - best_bid)

# Define bucket em USD
if bucket_mode == 'pct':
    raw_bucket_usd = mid * (bucket_bps / 10000.0)
elif bucket_mode == 'usd':
    raw_bucket_usd = float(bucket_usd_param)
elif bucket_mode == 'ticks':
    raw_bucket_usd = max(ticks_param * tick_size, tick_size)
else:
    raw_bucket_usd = mid * 0.0001

if nice_bucket:
    raw_bucket_usd = nice_125(raw_bucket_usd, mode=nice_mode)

bucket_ticks = max(1, int(np.floor(raw_bucket_usd / tick_size)))
bucket_usd = bucket_ticks * tick_size

# Aritmética em ticks para evitar erro de ponto flutuante
df_orderbook['price_ticks'] = np.rint(df_orderbook['price'] / tick_size).astype(int)
bucket_idx = df_orderbook['price_ticks'] // bucket_ticks
df_orderbook['bucket_floor_ticks'] = bucket_idx * bucket_ticks
df_orderbook['bucket_floor_price'] = df_orderbook['bucket_floor_ticks'] * tick_size
df_orderbook['price_range_label'] = df_orderbook['bucket_floor_price'].apply(
    lambda x: f"{x:.{label_decimals}f} - {(x + bucket_usd):.{label_decimals}f}"
)

# Agrega
agg = df_orderbook.groupby(
    ['bucket_floor_ticks','price_range_label','side']
).agg(
    Total_Quantity=('quantity','sum'),
    Min_Price=('price','min'),
    Max_Price=('price','max'),
    Order_Count=('price','count'),
).reset_index()

# Completa buckets vazios (opcional)
if show_empty_buckets:
    all_buckets = np.sort(agg['bucket_floor_ticks'].unique())
    all_sides = ['BID','ASK']
    complete_idx = pd.MultiIndex.from_product([all_buckets, all_sides], names=['bucket_floor_ticks','side'])
    complete = pd.DataFrame(index=complete_idx).reset_index()
    complete = complete.merge(agg.drop(columns=['price_range_label']), on=['bucket_floor_ticks','side'], how='left')
    complete['bucket_floor_price'] = complete['bucket_floor_ticks'] * tick_size
    complete['price_range_label'] = complete['bucket_floor_price'].apply(
        lambda x: f"{x:.{label_decimals}f} - {(x + bucket_usd):.{label_decimals}f}"
    )
    complete['Total_Quantity'] = complete['Total_Quantity'].fillna(0)
    complete['Order_Count'] = complete['Order_Count'].fillna(0).astype(int)
    agg = complete[['price_range_label','side','Total_Quantity','Min_Price','Max_Price','Order_Count','bucket_floor_ticks']]

# Ordena: ASKs (baixo->alto), depois BIDs (alto->baixo)
asks_df = agg[agg['side']=='ASK'].sort_values('bucket_floor_ticks', ascending=True)
bids_df = agg[agg['side']=='BID'].sort_values('bucket_floor_ticks', ascending=False)
liquidity_consolidated_rpi = pd.concat([asks_df, bids_df], ignore_index=True)
liquidity_consolidated_rpi = liquidity_consolidated_rpi[['price_range_label','side','Total_Quantity','Min_Price','Max_Price','Order_Count']]

print("\n" + "="*80)
print(f"📊 RPI LIQUIDEZ POR BUCKET (bucket≈{bucket_usd:.{label_decimals}f} USD | mode={bucket_mode}, bps={bucket_bps})")
print("="*80)
print(liquidity_consolidated_rpi.to_string(index=False))

print("\n" + "="*80)
print("📈 RESUMO (RPI)")
print("="*80)
total_bids = df_bids['quantity'].sum()
total_asks = df_asks['quantity'].sum()
print(f"Total BID: {total_bids:.6f}")
print(f"Total ASK: {total_asks:.6f}")
print(f"Melhor BID: {best_bid:.{label_decimals}f}")
print(f"Melhor ASK: {best_ask:.{label_decimals}f}")
print(f"Spread: {(best_ask-best_bid):.{label_decimals}f}")

liquidity_consolidated_rpi


📊 RPI LIQUIDEZ POR BUCKET (bucket≈10.00 USD | mode=pct, bps=1.0)
    price_range_label side  Total_Quantity  Min_Price  Max_Price  Order_Count
111570.00 - 111580.00  ASK           7.684   111577.3   111579.4            9
111580.00 - 111590.00  ASK           7.144   111580.0   111587.7           41
111570.00 - 111580.00  BID           0.485   111570.0   111577.2           13
111560.00 - 111570.00  BID           1.898   111560.0   111569.4           34
111550.00 - 111560.00  BID           0.115   111559.0   111559.8            3

📈 RESUMO (RPI)
Total BID: 2.498000
Total ASK: 14.828000
Melhor BID: 111577.20
Melhor ASK: 111577.30
Spread: 0.10


,price_range_label,side,Total_Quantity,Min_Price,Max_Price,Order_Count
0,111570.00 - 111580.00,ASK,7.684,111577.3,111579.4,9
1,111580.00 - 111590.00,ASK,7.144,111580.0,111587.7,41
2,111570.00 - 111580.00,BID,0.485,111570.0,111577.2,13
3,111560.00 - 111570.00,BID,1.898,111560.0,111569.4,34
4,111550.00 - 111560.00,BID,0.115,111559.0,111559.8,3


In [33]:
import pandas as pd
import numpy as np

# Assuma:
# - liquidity_consolidated_full: DF do book completo (colunas: price_range_label, side, Total_Quantity, ...)
# - liquidity_consolidated_rpi:  DF do RPI          (mesmas colunas)
# - order_book: resposta do /v5/market/orderbook (completo)
# - data:       resposta do /v5/market/rpi_orderbook (RPI)

def bucket_lb_from_label(s: str) -> float:
    return float(s.split(' - ')[0])

full_df = liquidity_consolidated_full.copy()
rpi_df  = liquidity_consolidated_rpi.copy()

full_df['bucket_lb'] = full_df['price_range_label'].apply(bucket_lb_from_label)
rpi_df['bucket_lb']  = rpi_df['price_range_label'].apply(bucket_lb_from_label)

cmp_df = full_df[['bucket_lb','side','Total_Quantity']].rename(columns={'Total_Quantity':'Full_Qty'}) \
    .merge(
        rpi_df[['bucket_lb','side','Total_Quantity']].rename(columns={'Total_Quantity':'RPI_Qty'}),
        on=['bucket_lb','side'], how='outer'
    ).fillna(0)

cmp_df['RPI_Ratio'] = np.where(cmp_df['Full_Qty'] > 0, cmp_df['RPI_Qty'] / cmp_df['Full_Qty'], np.nan)

asks = cmp_df[cmp_df['side']=='ASK'].sort_values('bucket_lb', ascending=True)
bids = cmp_df[cmp_df['side']=='BID'].sort_values('bucket_lb', ascending=False)
comparison = pd.concat([asks, bids], ignore_index=True)

print("\n=== Comparação RPI vs Book por bucket ===")
print(comparison.to_string(index=False))

# Improvement no top-of-book
full_best_bid = max(float(p) for p,_ in order_book['result']['b'])
full_best_ask = min(float(p) for p,_ in order_book['result']['a'])
rpi_best_bid  = max(float(p) for p,*_ in data['result']['b'])
rpi_best_ask  = min(float(p) for p,*_ in data['result']['a'])

print("\n=== Price Improvement (Top-of-Book) ===")
print(f"Best bid improvement (RPI - Full): {rpi_best_bid - full_best_bid:.2f} USD")
print(f"Best ask improvement (Full - RPI): {full_best_ask - rpi_best_ask:.2f} USD")


=== Comparação RPI vs Book por bucket ===
 bucket_lb side  Full_Qty  RPI_Qty  RPI_Ratio
  111570.0  ASK     7.674    7.684   1.001303
  111580.0  ASK     8.541    7.144   0.836436
  111590.0  ASK     8.928    0.000   0.000000
  111570.0  BID     0.489    0.485   0.991820
  111560.0  BID     2.195    1.898   0.864692
  111550.0  BID    11.543    0.115   0.009963

=== Price Improvement (Top-of-Book) ===
Best bid improvement (RPI - Full): -10.50 USD
Best ask improvement (Full - RPI): 10.50 USD


In [34]:
public_trade_history = cliente.get_public_trade_history(category='linear', symbol=simbolo, limit=1000)
public_trade_history

{'retCode': 0,
 'retMsg': 'OK',
 'result': {'category': 'linear',
  'list': [{'execId': 'f9cc4f14-63dc-5dd5-ad05-a9bb1c68968f',
    'symbol': 'BTCUSDT',
    'price': '111577.20',
    'size': '0.004',
    'side': 'Sell',
    'time': '1761422022311',
    'isBlockTrade': False,
    'isRPITrade': False,
    'seq': '470630372141'},
   {'execId': '44a17f8f-fbd7-5826-bb41-a1faf37d3ede',
    'symbol': 'BTCUSDT',
    'price': '111577.30',
    'size': '0.010',
    'side': 'Buy',
    'time': '1761422019372',
    'isBlockTrade': False,
    'isRPITrade': False,
    'seq': '470630369930'},
   {'execId': '9c10f912-9607-5123-84f4-7a7b24233457',
    'symbol': 'BTCUSDT',
    'price': '111577.40',
    'size': '0.001',
    'side': 'Sell',
    'time': '1761422013293',
    'isBlockTrade': False,
    'isRPITrade': False,
    'seq': '470630357864'},
   {'execId': 'b294e776-0f29-5673-ba3a-0ad87cce6388',
    'symbol': 'BTCUSDT',
    'price': '111577.40',
    'size': '0.053',
    'side': 'Sell',
    'time': '176

In [35]:
import pandas as pd
import numpy as np

# 1) Parse e limpeza
trades_raw = public_trade_history['result']['list']
trades = pd.DataFrame(trades_raw, dtype=object)

if trades.empty:
    raise ValueError("Sem trades na resposta.")

trades = trades.drop_duplicates(subset=['execId']).copy()
trades['price'] = trades['price'].astype(float)
trades['size'] = trades['size'].astype(float)
trades['side'] = trades['side'].astype(str)
trades['isRPITrade'] = trades['isRPITrade'].astype(bool)
trades['isBlockTrade'] = trades['isBlockTrade'].astype(bool)
trades['ts'] = pd.to_datetime(trades['time'].astype('int64'), unit='ms', utc=True)

# Lado agressor (+1 Buy, -1 Sell) e quantidade assinada
trades['aggr_sign'] = np.where(trades['side'].str.upper()=='BUY', 1, -1)
trades['signed_qty'] = trades['aggr_sign'] * trades['size']

# 2) Mid e distâncias (usa order_book se existir; senão, fallback)
try:
    best_bid = max(float(p) for p,_ in order_book['result']['b'])
    best_ask = min(float(p) for p,_ in order_book['result']['a'])
    mid = (best_bid + best_ask) / 2.0
except Exception:
    mid = trades['price'].median()

trades['dist_to_mid_bps'] = (trades['price'] - mid) / mid * 1e4

# 3) Agregação temporal (1s)
trades['ts_1s'] = trades['ts'].dt.floor('1s')

def _vwap(g):
    v = g['size'].sum()
    return (g['price'] * g['size']).sum() / v if v > 0 else np.nan

tb = trades.groupby('ts_1s').agg(
    total_vol=('size','sum'),
    buy_vol=('size', lambda s: trades.loc[s.index][trades.loc[s.index,'side'].str.upper()=='BUY']['size'].sum()),
    sell_vol=('size', lambda s: trades.loc[s.index][trades.loc[s.index,'side'].str.upper()=='SELL']['size'].sum()),
    buy_count=('side', lambda s: (trades.loc[s.index,'side'].str.upper()=='BUY').sum()),
    sell_count=('side', lambda s: (trades.loc[s.index,'side'].str.upper()=='SELL').sum()),
    vwap=('price', lambda _: _vwap(trades.loc[_ .index])),
    rpi_share=('isRPITrade', 'mean'),
    block_vol=('size', lambda s: trades.loc[s.index][trades.loc[s.index,'isBlockTrade']]['size'].sum()),
).reset_index()

tb['net_flow'] = (tb['buy_vol'] - tb['sell_vol']).fillna(0.0)
tb['trade_rate'] = (tb['buy_count'] + tb['sell_count']).astype(int)

# 4) Buckets de preço dinâmicos (1–2–5 arredondado e alinhado ao tick)
def infer_tick_from_prices(p_series):
    # usa número máximo de casas decimais observadas
    dec = max((str(x)[::-1].find('.') if '.' in str(x) else 0) for x in p_series)
    return 10**(-dec) if dec > 0 else 1.0

def nice_125(x, mode='floor'):
    if x <= 0: return x
    exp = np.floor(np.log10(x)); frac = x / (10**exp)
    if mode=='floor':
        n = 5 if frac >= 5 else (2 if frac >= 2 else 1)
    elif mode=='ceil':
        n = 1 if frac <= 1 else (2 if frac <= 2 else (5 if frac <= 5 else 10))
    else:
        n = 1 if frac < 1.5 else (2 if frac < 3.5 else (5 if frac < 7.5 else 10))
    return n * (10**exp)

tick = infer_tick_from_prices(trades['price'])
label_decimals = max(0, int(round(-np.log10(tick)))) if tick < 1 else 0

# bucket por bps do mid, arredondado 1–2–5 e alinhado ao tick
bps = 1.0  # ajuste conforme o ativo (0.5–2 bps majors; 2–10 bps alts)
raw_bucket = mid * (bps / 1e4)
raw_bucket = nice_125(raw_bucket, mode='floor')
bucket_ticks = max(1, int(np.floor(raw_bucket / tick)))
bucket_usd = bucket_ticks * tick

# indexação em ticks para robustez numérica
trades['price_ticks'] = np.rint(trades['price'] / tick).astype(int)
trades['bucket_floor_ticks'] = (trades['price_ticks'] // bucket_ticks) * bucket_ticks
trades['bucket_floor_price'] = trades['bucket_floor_ticks'] * tick
trades['price_range_label'] = trades['bucket_floor_price'].apply(
    lambda x: f"{x:.{label_decimals}f} - {(x + bucket_usd):.{label_decimals}f}"
)

# mapeia lado do trade -> lado consumido do book
trades['book_side'] = np.where(trades['side'].str.upper()=='BUY', 'ASK', 'BID')

trade_buckets = trades.groupby(['price_range_label','book_side']).agg(
    Traded_Qty=('size','sum'),
    Min_Trade_Price=('price','min'),
    Max_Trade_Price=('price','max'),
    Trade_Count=('price','count'),
    Buy_Share=('side', lambda s: (trades.loc[s.index,'side'].str.upper()=='BUY').mean()),
    RPI_Share=('isRPITrade','mean'),
).reset_index()

# ordenação: ASKs (menor->maior), depois BIDs (maior->menor)
def lb(s): return float(s.split(' - ')[0])
trade_buckets['lb'] = trade_buckets['price_range_label'].apply(lb)
asks = trade_buckets[trade_buckets['book_side']=='ASK'].sort_values('lb', ascending=True)
bids = trade_buckets[trade_buckets['book_side']=='BID'].sort_values('lb', ascending=False)
trade_buckets_sorted = pd.concat([asks, bids], ignore_index=True).drop(columns=['lb'])

# 5) (Opcional) Cruzamento com liquidez do book por bucket
# Requer um DF 'liquidity_consolidated' no formato já gerado antes
try:
    lc = liquidity_consolidated_full.rename(columns={'Total_Quantity':'Book_Liquidity'})
    comp = trade_buckets.merge(
        lc[['price_range_label','side','Book_Liquidity']].rename(columns={'side':'book_side'}),
        on=['price_range_label','book_side'], how='left'
    )
    comp['Trade/Liquidity'] = np.where(comp['Book_Liquidity']>0, comp['Traded_Qty']/comp['Book_Liquidity'], np.nan)
    comparison_trade_vs_book = comp
except Exception:
    comparison_trade_vs_book = None

# Saídas principais:
# - trades: trades normalizados (linha-a-linha)
# - tb: métricas por segundo (fluxo)
# - trade_buckets_sorted: volume de trades por faixa de preço e lado do book
# - comparison_trade_vs_book: (opcional) razão trade/liquidez por faixa
print(f"Bucket ≈ {bucket_usd:.{label_decimals}f} USD (bps={bps}) | tick={tick} | mid={mid:.{label_decimals}f}")
tb.tail(5), trade_buckets_sorted.head(10), comparison_trade_vs_book.head(10) if comparison_trade_vs_book is not None else None

Bucket ≈ 10.0 USD (bps=1.0) | tick=0.1 | mid=111577.2


(                       ts_1s  total_vol  buy_vol  sell_vol  buy_count  \
 95 2025-10-25 19:53:30+00:00      0.026    0.026     0.000          6   
 96 2025-10-25 19:53:32+00:00      0.039    0.009     0.030          2   
 97 2025-10-25 19:53:33+00:00      0.281    0.000     0.281          0   
 98 2025-10-25 19:53:39+00:00      0.010    0.010     0.000          1   
 99 2025-10-25 19:53:42+00:00      0.004    0.000     0.004          0   
 
     sell_count           vwap  rpi_share  block_vol  net_flow  trade_rate  
 95           0  111577.500000        0.0        0.0     0.026           6  
 96           5  111577.423077        0.0        0.0    -0.021           7  
 97           8  111577.400000        0.0        0.0    -0.281           8  
 98           0  111577.300000        0.0        0.0     0.010           1  
 99           1  111577.200000        0.0        0.0    -0.004           1  ,
      price_range_label book_side  Traded_Qty  Min_Trade_Price  \
 0  111530.0 - 111540.0  